In [1]:
%load_ext autoreload
%autoreload 2
%load_ext line_profiler
%matplotlib inline

import sys
import os
import yaml
import itertools
sys.path.append('../')
from utils.utils import ModelLoader,DepLoader
from matplotlib import pyplot as plt
from timeit import timeit  

In [2]:
with open('../config/rule_shortdeck.yaml') as fhdl:
    cfg = yaml.load(fhdl)

/root/anaconda3/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:2: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  


In [3]:
env = DepLoader(cfg)


Loading abstraction file: PHS1_PREFLOP.txt  100.00 % [==================================================>] 630/630 	 used:0s eta:0 s
Loading abstraction file: PHS1_FLOP_300buckets.txt  100.08 % [==================================================>] 3772769/3769920 	 used:6s eta:0 s
Loading abstraction file: PHS1_TURN_300buckets.txt  100.10 % [==================================================>] 29245216/29216880 	 used:54s eta:0 s
Loading abstraction file: PHS1_RIVER_300buckets.txt  100.10 % [==================================================>] 175476301/175301280 	 used:332s eta:0 s

# 查看游戏总状态数

In [109]:
hiss = []
roots = []
nonterminals = []
actions = []
def count_dealcard(root,depth=0):
    if root.children is not None:
        num = 0
        actions.append(root.bet_history)
        for one_child in root.children.values():
            num += count_dealcard(one_child)
        return num
    else:
        if 'terminal' in root.to_string():
            hiss.append(root.bet_history)
            roots.append(root)
        else:
            nonterminals.append(root.bet_history)
        return 0 if 'terminal' in root.to_string() else 1

In [110]:
count_dealcard(env.tree_builder.root)

47662

In [111]:
len(actions)

61385

In [112]:
%timeit count_dealcard(env.tree_builder.root)

942 ms ± 20.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [113]:
#%lprun -f count_dealcard count_dealcard(env.tree_builder.root)

In [114]:
len(hiss)

428958

# 查看叶子节点的bet history

In [115]:
index = 5000 + 1
print(hiss[index])
print(roots[index].committed)

['call', 'check', 'check', 'bet_1', 'raise_2', 'raise_5', 'call', 'check', 'bet_7', 'raise_21', 'raise_84', 'call', 'check', 'bet_2', 'fold']
[100.0, 98.0]


# 查看card abstraction 工作状态

In [12]:
cards_sampled = env.card_sampler.sample_card(4)
print(cards_sampled)

['Qc', '9s', '6c', '7h']


In [13]:
env.card_abstraction.abstract(cards_sampled)

'Qc9s6c7h'

In [20]:
# TODO 写strategy 和trainer,修改abstraction算法
# TODO 每层都需要用不同的action choice

In [5]:
from pyemd import emd
import numpy as np

In [26]:
first_histogram = np.array([0.0, 5.0,1.0])
second_histogram = np.array([2.0, 4.0,0.0])
distance_matrix = np.array([[0.0, 1.0, 2.0],
                            [1.0, 0.0, 1.0],
                            [2.0, 1.0, 0.0]])
emd(first_histogram, second_histogram, distance_matrix)

2.999994

In [94]:
!free

              total        used        free      shared  buff/cache   available
Mem:       61848116    41441368    19441124        5324      965624    19921324
Swap:             0           0           0


In [95]:
!mkdir ../data/abstractions

In [14]:
public = ('6c', '6d', '7c', '7d', '9c')
private = ('8d','Jc')
env.card_abstraction.abstract((private,public))

8d-Jc:6c-6d-7c-7d-9c


9